In [ ]:
import yfinance as yf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

TICKER = "SPY"

INTERVAL = "1h"
if INTERVAL == "1h":
    PERIOD = "720d"
else:
    PERIOD = "max"

STRATEGY = ["Volume"]

SHIFT = 5 

LOOKBACK = 10000

def get_data ():

